In [1]:
#DB에서 영화cd,국적, 장르, 제작사에 따른 매출액 예측 할수 있는 데이터 프래임 만들기

In [206]:
import pandas as pd
from datetime import datetime
from DbConn import *
import numpy as np
from sklearn.model_selection import train_test_split

In [207]:
db= DbConn()
#GENRENM,COMPANYCD1

In [208]:
# sql = '''
# SELECT m.MOVIECD, m.MOVIENM, m.NATIONS, m.GENRENM, m.COMPANYCD1, b.OPENDT, b.DAILYDATE2, b.AUDIACC
# FROM MOVIEDETAIL m JOIN (
#    SELECT MOVIECD, OPENDT, DAILYDATE2, AUDIACC FROM BOXOFFICE WHERE DAILYDATE2 = OPENDT +60
# ) b ON m.MOVIECD = b.MOVIECD 
# '''

In [209]:
# sql2 = '''
# SELECT m.MOVIECD, m.MOVIENM, m.NATIONS, m.GENRENM, m.COMPANYCD1, b.OPENDT, b.DAILYDATE2, b.AUDIACC
# FROM MOVIEDETAIL m JOIN (
#    SELECT * FROM BOXOFFICE 
#    WHERE AUDIACC IN (SELECT MAX(AUDIACC) FROM BOXOFFICE GROUP BY MOVIECD)
# ) b ON m.MOVIECD = b.MOVIECD  
# '''

In [210]:
sql3 ='''
select 
CASE WHEN audiacc > 10000000 THEN 1 
when audiacc > 2600000 then 2
when audiacc > 680000 then 3
when audiacc > 130000 then 4
when audiacc > 30000 then 5
ELSE 6 END AUDIACC_GRADE, origin.*
from (
    SELECT b.*, m.MOVIENM , m.SHOWTM , m.NATIONS , m.GENRENM , m.DIRECTORS , m.AUDITS , m.COMPANYCD1
    FROM MOVIEDETAIL m JOIN (
    SELECT MOVIECD , MAX(AUDIACC) as audiacc
    FROM BOXOFFICE b2 GROUP BY MOVIECD 
    ) b ON m.MOVIECD = b.MOVIECD
) origin
'''

In [211]:
#dataFrame 객체 가져오기
query_result_df=pd.read_sql(sql3,db.connection)

In [212]:
db.disconnect()

In [213]:
query_result_df.keys()

Index(['AUDIACC_GRADE', 'MOVIECD', 'AUDIACC', 'MOVIENM', 'SHOWTM', 'NATIONS',
       'GENRENM', 'DIRECTORS', 'AUDITS', 'COMPANYCD1'],
      dtype='object')

In [214]:
X = query_result_df.drop(['AUDIACC_GRADE','MOVIECD','AUDIACC','MOVIENM','AUDITS','DIRECTORS','SHOWTM','AUDIACC'],axis=1)

In [215]:
y=query_result_df['AUDIACC_GRADE']
# y.head()

In [216]:
conditions = [
    (X['NATIONS'] == '한국'), # S
    (X['NATIONS'] == '미국'), # A
    (X['NATIONS'] == '일본') # A
]
nation_code = [1,2,3]
X['nation_code'] = np.select(conditions, nation_code, default=4)
X = X.drop(['NATIONS'], axis=1)

In [217]:
X.keys()

Index(['GENRENM', 'COMPANYCD1', 'nation_code'], dtype='object')

In [218]:
conditions_genr = [
    (X['GENRENM'] == '애니메이션'),
    (X['GENRENM'] == '드라마'),
    (X['GENRENM'] == '액션'),
    (X['GENRENM'] == '코미디'),
    (X['GENRENM'] == '멜로/로맨스'),
    (X['GENRENM'] == '범죄'),
    (X['GENRENM'] == '공포(호러)'),
    (X['GENRENM'] == '스릴러'),
    (X['GENRENM'] == '다큐멘터리'),
    (X['GENRENM'] == '미스터리'),
    (X['GENRENM'] == '어드벤처'),
    (X['GENRENM'] == '판타지'),
    (X['GENRENM'] == '사극'),
    (X['GENRENM'] == 'SF'),
    (X['GENRENM'] == '가족'),
    (X['GENRENM'] == '전쟁'),
    (X['GENRENM'] == '뮤지컬')
]
genre_code = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

X['genre_code'] = np.select(conditions_genr, genre_code, default=18)
X = X.drop(['GENRENM'], axis=1)

In [219]:

# df = pd.DataFrame(X['GENRENM'])
# df.apply(pd.value_counts)
# # # df
# # df.groupby('GENRENM').count()


In [220]:
X=pd.get_dummies(X[['genre_code', 'COMPANYCD1', 'nation_code']],columns=['nation_code','genre_code'])

In [221]:
X.head(50)

,COMPANYCD1,nation_code_1,nation_code_2,nation_code_3,nation_code_4,genre_code_1,genre_code_2,genre_code_3,genre_code_4,genre_code_5,...,genre_code_9,genre_code_10,genre_code_11,genre_code_12,genre_code_13,genre_code_14,genre_code_15,genre_code_16,genre_code_17,genre_code_18
0,20122837,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20122837,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2016841,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20112452,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20000384,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,20161101,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,20100041,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,20141469,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,20100603,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,20101558,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [222]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=999)

In [223]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score

In [224]:
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [225]:
model.score(X_train,y_train)

0.7920883164673413

In [226]:
model.score(X_test,y_test)

0.3497854077253219